In [10]:
from io import BytesIO
from urllib import request
from keras_image_helper import create_preprocessor
import numpy as np

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [14]:
def preprocess_pytorch(X):
    # X: shape (1, 299, 299, 3), dtype=float32, values in [0, 255]
    X = X / 255.0

    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1)

    # Convert NHWC → NCHW
    # from (batch, height, width, channels) → (batch, channels, height, width)
    X = X.transpose(0, 3, 1, 2)

    # Normalize
    X = (X - mean) / std

    return X.astype(np.float32)


preprocessor = create_preprocessor(preprocess_pytorch, target_size=(200, 200))

In [16]:
URL = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'
X = preprocessor.from_url(URL)

In [17]:
X

array([[[[-1.0732939 , -1.0047948 , -1.0390444 , ..., -1.1075435 ,
          -1.0732939 , -1.210292  ],
         [-1.0561692 , -1.0219196 , -1.0047948 , ..., -1.0219196 ,
          -1.0561692 , -1.1760424 ],
         [-0.9534206 , -0.97054535, -0.9534206 , ..., -1.0047948 ,
          -1.1075435 , -1.2616662 ],
         ...,
         [-1.6726604 , -1.6726604 , -1.5185375 , ...,  1.7351657 ,
           1.649542  ,  1.7865399 ],
         [-1.6726604 , -1.6555356 , -1.6384108 , ...,  1.6837914 ,
           1.6152924 ,  1.7009162 ],
         [-1.6384108 , -1.7240345 , -1.6726604 , ...,  1.7009162 ,
           1.7351657 ,  1.6837914 ]],

        [[-0.21498597, -0.10994396, -0.10994396, ..., -0.495098  ,
          -0.477591  , -0.635154  ],
         [-0.19747896, -0.17997196, -0.16246496, ..., -0.512605  ,
          -0.547619  , -0.60014004],
         [-0.07492995, -0.12745096, -0.12745096, ..., -0.495098  ,
          -0.60014004, -0.687675  ],
         ...,
         [-1.1428571 , -1.1428571 

In [18]:
import onnxruntime as ort

onnx_model_path = "hair_classifier_v1.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

In [19]:
inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

In [20]:
result = session.run([output_name], {input_name: X})

In [21]:
result

[array([[0.08927515]], dtype=float32)]